In [ ]:
import sqlite3
import hashlib
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from constants import *
from utilities import get_node_info
from datetime import datetime

In [ ]:
## Paths for the server data files
APPINVENTOR_PATH = 'data/STORAGE'
SAVE_PATH = 'out_files/'
SAVE_DATA_PATH = os.path.join(SAVE_PATH, 'noPII')
APPINVENTOR_FILE_PATH = os.path.join(APPINVENTOR_PATH, '../ed_lc.out')
COURSE_SEQUENTIAL = os.path.join(COURSE_PATH, 'sequential/')
COURSE_VERTICAL = os.path.join(COURSE_PATH, 'vertical/')
COURSE_CHAPTER = os.path.join(COURSE_PATH, 'chapter/')

In [ ]:
def hash_username(n):
    """Return the digest of a hashed username using RSA's MD5 algorithm

    :param n: The username to hash
    :returns: The hashed usernames digest
    :rtype: String

    """
    return hashlib.md5(n.encode('utf-8')).hexdigest()

def hash_column(df_column):
    """Hash all usernames in a pandas dataframe column

    :param df_column: The column to hash the usernames of
    :returns: The column with its usernames hashed
    :rtype: pandas.DataFrame

    """
    return df_column.apply(lambda x: hash_username(x))

def edit_name(name):
    """Append "ed_" to the front of a name.

    :param name: The name to append to.
    :returns: The modified name
    :rtype: String

    """
    return 'ed_{}'.format(name)                                         

In [ ]:
# Load data
files = os.listdir(EDX_DATA)

print(*files, sep='\n')

id_map = pd.read_csv(os.path.join(EDX_DATA, ID_MAP_FILE_NAME))

print(id_map.info())

id_map['username'] = hash_column(id_map['username'])
id_map['mapped_username_on_alfa'] = hash_column(id_map['mapped_username_on_alfa'])

print(id_map.head(2))

# Save
id_map.to_csv(os.path.join(SAVE_DATA_PATH, edit_name(ID_MAP_FILE_NAME)))

In [ ]:
## Grades
grade_path = os.path.join(EDX_DATA, GRADE_FILE_NAME)
grades = pd.read_csv(grade_path)

print(grades.info())

grades['Username'] = hash_column(grades['Username'])

print(grades.Username.values[0])

# Save
grades.to_csv(os.path.join(SAVE_DATA_PATH, edit_name(GRADE_FILE_NAME)))


In [ ]:
## Users from sqlite
c = sqlite3.connect(os.path.join(APPINVENTOR_PATH, 'users.sqlite')).cursor()

appinventor_users = {}

for row in c.execute("SELECT uuid, email FROM users"):
    appinventor_users[row[0]] = hash_username(row[1])
    
print(list(appinventor_users.items())[:1])


In [ ]:
## AppInventor file creation
ai_df = pd.read_csv(APPINVENTOR_FILE_PATH, delimiter='\s+', parse_dates=[[5,6]])
ai_df = ai_df[['day_hour', 'bytes', 'path']]

print(ai_df.info())
print(ai_df.head(2))


In [ ]:
## Get appinventor usernames and uuids for the files
usernames = []
uuids = []
paths = []

for name, row in ai_df.iterrows():
    split = row.path.split('/')
    uuid = split[1]    

    uuids.append(uuid)

    try:
        n = appinventor_users[uuid]
    except KeyError:
        n = 'NA'

    usernames.append(n)

    if len(split) > 5 and split[5].startswith('ai_'):
        username = hash_username(split[5].split('ai_')[1])
        split[5] = username
    
    split = '/'.join(split)
    paths.append(split)        
    
ai_df['username'] = usernames
ai_df['uuid'] = uuids
ai_df['path'] = paths

print(ai_df.head(2))
print(ai_df.path.values[4])

# Save
ai_df.to_csv(os.path.join(SAVE_DATA_PATH, edit_name('appinventor_files.csv')))


In [ ]:
## Read clickstream data
clickstream_path = os.path.join(EDX_DATA, CLICKSTREAM_FILE_NAME) 
clickstreams_json = []

# For getting only a slice
N_LINES = 100000000
bad_lines = []

with open(clickstream_path, 'r') as fd:
    for i, line in enumerate(fd):
        try:
            clickstreams_json.append(json.loads(line))
        except json.JSONDecodeError as e:
           bad_lines.append((i, e))

        if i > N_LINES:
            break

print(len(clickstreams_json), len(bad_lines))


In [ ]:
clickstreams = pd.DataFrame(clickstreams_json)
clickstreams['username'] = hash_column(clickstreams['username'])

print(clickstreams.info())

print("""AppInventor users:{}\n
EdX users in student_profile-mini:{}\n
clickstream usernames:{}""".format(len(appinventor_users), 
                                   len(id_map), 
                                   clickstreams.username.nunique()))

# Save
clickstreams.to_csv(os.path.join(SAVE_DATA_PATH, edit_name(CLICKSTREAM_FILE_NAME)))


In [ ]:
## Who is in the mapping
unique_users = clickstreams.username.unique()
user_matches = {'click_id_map': [], 'id_map_app': []}

for row in id_map.iterrows():
    if row[1].username in unique_users:
        user_matches['click_id_map'].append(row)

        if row[1].mapped_username_on_alfa in ai_df.username.values:
            user_matches['id_map_app'].append(row)
        
print('Clickstream and id_map matches: {}'.format(len(user_matches['click_id_map'])))
print('Clickstream and id_map and AppInventor matches: {}'.format(len(user_matches['id_map_app'])))


- Link activity at EdX and AppInventor
 - Identify one user
 - Plot EdXs use and AppInventor use on a timeline
 - Visits to and from AppInventor
 
 Assumptions: the file creation time on appinventor and the click time on EdX is used
 
 TODO create PII files

In [ ]:
EDX_TYPE = 1
AI_TYPE = 2

# Transition types determined by access of either the EDX server or
# the AppInventor server and then the subsequent access of either 
# of the two
TRANSITION_TYPES = {
    (EDX_TYPE, EDX_TYPE): 3,
    (EDX_TYPE, AI_TYPE):  4,
    (AI_TYPE, EDX_TYPE):  5,
    (AI_TYPE, AI_TYPE):   6,
}

def get_empty_transitions():
    """Get an empty dictionary to hold all transitions

    Return a dictionary to hold each transitions type and duration as lists.

    :returns: The empty dictionary of lists.
    :rtype: Dictionary of lists.

    """
    return {'type': [], 'duration': []}

def get_ai_user_dates(ai_user):
    """Get the times a user interacted with AppInventor

    :param ai_user: The user
    :returns: A list of times the user interacted with the server
    :rtype: List

    """
    return ai_df[ai_df['username'] == ai_user].day_hour.dt.tz_localize('UTC').sort_values()

def get_clickstream_user_dates_from_appinventor(ai_user):
    edx_user = id_map[id_map['mapped_username_on_alfa'] == ai_user].username.values[0]
    click_times = clickstreams[clickstreams['username'] == edx_user]
    
    # Filter name TODO correct
    click_times = click_times[pd.notnull(click_times['name'])]

    click_times['time'] = pd.to_datetime(click_times['time'])

    return click_times.time.sort_values()
    
def combine_ai_and_clickstream_dates(ai_dates, click_times):
    edxs = [EDX_TYPE] * len(click_times)
    edx_vals = list(zip(click_times, edxs))

    ais = [AI_TYPE] * len(ai_dates)
        ai_vals = list(zip(ai_dates, ais))
    combined = sorted(ai_vals + edx_vals, key=lambda x: x[0])
    
    return np.array(combined)

def get_ai_edx_transitions(clicks_and_files):
    transitions = get_empty_transitions()
    # TODO should be built in functions for this...
    for i in range(1, clicks_and_files.shape[0]):
        # TODO do not make your own categories
        t_type = TRANSITION_TYPES[(clicks_and_files[i-1][1], clicks_and_files[i][1])]
        transitions['type'].append(t_type)
        duration = clicks_and_files[i][0] - clicks_and_files[i-1][0]
        transitions['duration'].append(duration)

    return transitions

In [ ]:
sample_alfa_user = hash_username('user@em.ail')
dates = get_ai_user_dates(sample_alfa_user)

print(dates[:3])

plt.plot(range(len(dates)), dates, linestyle='None', marker='o')


In [ ]:
clicks = get_clickstream_user_dates_from_appinventor(sample_alfa_user)
combined = combine_ai_and_clickstream_dates(dates, clicks)

print(combined)

plt.plot(combined[:,1], combined[:,0], linestyle='None', marker='o')


In [ ]:
transitions = get_ai_edx_transitions(combined)
display(transitions)
time_unit = 's'
times = [int(_ / np.timedelta64(1, time_unit)) for _ in transitions['duration']] 

plt.scatter(transitions['type'], times)
plt.xlabel('State')
plt.ylabel('Duration({})'.format(time_unit))


In [ ]:
def set_transition(user_id, transitions):
    sample_alfa_user = user_id.mapped_username_on_alfa

    alfa_dates = get_ai_user_dates(sample_alfa_user)
    edx_dates = get_clickstream_user_dates_from_appinventor(sample_alfa_user)
    ai_edx_dates = combine_ai_and_clickstream_dates(alfa_dates, edx_dates)

    transitions = get_ai_edx_transitions(ai_edx_dates)

    for key, value in transitions.items():
        all_transitions[key].extend(value)


all_transitions = get_empty_transitions()

for _, user_id in user_matches['id_map_app']:
    set_transition(user_id, all_transitions)

print(len(all_transitions['type']))

df = pd.DataFrame(all_transitions)
df['duration'] = df['duration'].apply(lambda x: int(x / np.timedelta64(1, time_unit))) 

print(df.head())


In [ ]:
print(df[df['type'] == 6].duration.nunique())

In [ ]:
for name, group in df.groupby(by='type'):
    duration = group[group['duration'] < 100]
    sns.distplot(duration.duration, label=str(name), kde=False)

plt.legend()


In [ ]:
sns.boxplot(x='type', y='duration', data=df)

In [ ]:
df['ed_duration'] = df[df['duration'] < 600].duration
sns.boxplot(x='type', y='ed_duration', data=df)

In [ ]:
# Grades
non_zero = grades[grades['Grade'] > 0]

print(non_zero.shape)

grade_users_in_appinventor = []
        
non_zero_grade_usernames = non_zero.Username.values

for alfa_name in user_matches['id_map_app']:
    username = alfa_name[1].username

    if username in non_zero_grade_usernames:
        grade_users_in_appinventor.append(username)

print(len(grade_users_in_appinventor))


In [ ]:
all_transitions = get_empty_transitions()

for _, user_id in user_matches['id_map_app']:
    if user_id.username in grade_users_in_appinventor:
        set_transition(user_id, all_transitions)

print(len(all_transitions['type']))

df = pd.DataFrame(all_transitions)
df['duration'] = df['duration'].apply(lambda x: int(x / np.timedelta64(1, time_unit))) 
df['ed_duration'] = df[df['duration'] < 1000].duration

sns.boxplot(x='type', y='ed_duration', data=df)